In [ ]:
!pip install datasets evaluate
!pip install transformers==4.28.0

In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
id2label = {0: "Justice Breyer", 1: "Justice Ginsburg", 2: "Justice Kennedy", 3: "Justice O'Connor", 
            4: "Justice Rehnquist", 5: "Justice Scalia",6: "Justice Souter", 7: "Justice Stevens", 8: "Justice Thomas"}
label2id = {"Justice Breyer": 0, "Justice Ginsburg": 1, "Justice Kennedy": 2, "Justice O'Connor":3,
            "Justice Rehnquist":4, "Justice Scalia":5, "Justice Souter": 6, "Justice Stevens":7, "Justice Thomas":8}

In [ ]:
# full code
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
import torch.nn as nn
import torch
from transformers import AutoTokenizer
from transformers import DataCollatorWithPadding
import evaluate
import numpy as np
from datasets import load_dataset

court = load_dataset('raminass/scotus', use_auth_token=True)

arch = 'legal-bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(f"nlpaueb/{arch}")
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

tokenized_court = court.map(preprocess_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


accuracy = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

model = AutoModelForSequenceClassification.from_pretrained(
    f"nlpaueb/{arch}", num_labels=9, id2label=id2label, label2id=label2id,
    # hidden_dropout_prob = 0.4, attention_probs_dropout_prob = 0.2
)

# model.classifier = nn.Sequential(
#             nn.Linear(512, 100),
#             nn.ReLU(),
#             nn.Dropout(0.3),
#             nn.Linear(100, 50),
#             nn.ReLU(),
#             nn.Dropout(0.3),
#             nn.Linear(50, 9),
#         )

for param in model.bert.parameters():
    param.requires_grad = False


device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

training_args = TrainingArguments(
    output_dir=f"{arch}",
    logging_dir=f"{arch}/runs/feature-based-dflt-classifier",
    learning_rate=2e-5, #as in bert paper
    per_device_train_batch_size=16, # in legal-bert 256 however no memory here
    per_device_eval_batch_size=16, # in legal-bert 256 however no memory here
    num_train_epochs=500,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_court["train"],
    eval_dataset=tokenized_court["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

trainer.push_to_hub()

  0%|          | 0/2 [00:00<?, ?it/s]

Some weights of the model checkpoint at nlpaueb/legal-bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification wer

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.465361,0.499532
2,1.803700,1.025699,0.670412
3,1.008300,0.899840,0.701779
4,0.492300,0.823287,0.738764
5,0.186900,0.557434,0.832865
6,0.186900,0.721242,0.805243
7,0.059000,0.638023,0.841292
8,0.031800,0.741393,0.837079
9,0.013400,0.731718,0.847846
10,0.011800,1.109460,0.783240


Several commits (2) will be pushed upstream.
